### 1) Duplicate content detection on Screaming Frog Data With Shingling

* pip install nltk
   * After installing open python and type:
   * import nltk
   * nltk.download()
   * Load stopwords corpus.
* pip install mmh3
* pip install pandas
* pip install matplotlib

### 2) Here we import the required libraries
If you get an error here most likely you don't have the right version of Python, or one of the modules is not installed. Python should tell you you are missing a module and you can most often just type "!pip install [modulename]"

In [1]:
import mmh3
from nltk import ngrams
import pandas
import random
import matplotlib.pyplot as plt

In [2]:
list(df.columns.values)

NameError: name 'df' is not defined

### 3) Load in the data. 
The data can be loaded from Screeaming Frog > internal_html.csv export after crawling a site with an extraction set to pull all text from the main content body element using XPATH or CSS Selectors. If loaded correctly you should see the columns and top five rows below when running this cell. You can use another tool to pull site content, but will need to use csv format and ensure the content column is named "Column 1" and the associated URL column is named "Address".


In [3]:
in_file = "Data 3000 for Shingling.csv"
df = pandas.read_csv(in_file)
df = df[df['Body'] == df['Body']]
df.reset_index(drop=True, inplace=True)
df.head()

,ReviewID,MemberID,Verified,RealName,ReviewOrder,ProductID,Date,Rating,HelpfulFeedbacks,Feedbacks,Title,Body
0,600982,A3S8OV1ARSIS1,0,1,2,B0002Y5WZC,9/4/2007,5,0,3,Nice camera bag,This is a nice bag. It was too big for my need...
1,600960,A2T036DGT1BGY1,0,1,1,B0002Y5YFK,1/31/2007,1,0,14,Never received it.,I waited and waited for this item. One email ...
2,600760,A2A10ZSC2RH4RG,0,1,1,B0002Y7GGA,10/15/2004,5,0,0,HTML 4.01 Programming Curriculum CBT Training CDs,Course Overview This curriculum presents learn...
3,600759,A2A10ZSC2RH4RG,0,1,1,B0002Y7WBO,10/15/2004,5,0,0,Java Enterprise Connectivity Curriculum CBT Tr...,"Trunc, Course Overview This curriculum introdu..."
4,600758,A2A10ZSC2RH4RG,0,1,1,B0002Y8HH2,10/19/2004,5,0,0,Java Servlets Curriculum CBT Training CDs,Course Overview This curriculum introduces the...


### 5) These are functions that we will use.
Code for shingling taken from https://github.com/steven-s/text-shingles

In [4]:
def generate_random_seeds(n, seed=5):
    random.seed(seed)
    return random.sample(range(1, n+1), n)

def jaccard_similarity(set_a, set_b):
    return len(set_a.intersection(set_b)) / len(set_a.union(set_b))


class ShingledText:
    def __init__(self, text, random_seed=5, shingle_length=5, minhash_size=200):
        split_text = text.split()
        if len(split_text) < shingle_length:
            raise ValueError(u'input text is too short for specified shingle length of {}'.format(shingle_length))

        self.minhash = []
        self.shingles = ngrams(split_text, shingle_length)

        for hash_seed in generate_random_seeds(minhash_size, random_seed):
            min_value = float('inf')
            for shingle in ngrams(split_text, shingle_length):
                value = mmh3.hash(' '.join(shingle), hash_seed)
                min_value = min(min_value, value)
            self.minhash.append(min_value)

    def similarity(self, other_shingled_text):
        return jaccard_similarity(set(self.minhash), 
                set(other_shingled_text.minhash))
    
def apply_shingled(row, df, df_shingled):
    
    idx = row.name
    high = 0.0
    match = ""
    start = 0.0
    
    for i, s in enumerate(df_shingled):
        if i > len(df):
            print('i',i)
        if idx > len(df):
            print('idx',idx)
                
        if not i == idx:
            sim = df_shingled[idx].similarity(df_shingled[i])
            if sim > high:
                high = sim
                match = str(df['Body'][i])
    
    row['Sim Score'] = high
    row['Sim Match'] = match
        
    return row

    

### 6) Build a list of shingled text
We only want to do this one time as it takes time and we will have to re-iterate over every row to find the closest match.

In [ ]:
df_shingled = []

# Build content shingles list
for i, row in df.iterrows():
    
    text = row['Body']
    
    if isinstance(text, str) and len(text.split()) > 5:  
        df_shingled.append(ShingledText(text))
    else:
        df_shingled.append(False)
        

    

### 9) Apply similarity values and most similar url to dataframe

In [ ]:
df_comp = df.apply(apply_shingled, args=(df, df_shingled), axis=1)
df_comp.head(5)

### 10) Visualize your duplicate content. >.6 Should be reviewed.  1 are exact duplicates.

In [ ]:
import numpy as numpy

In [ ]:
sims = df_comp['Sim Score'].tolist()
a = numpy.histogram(sims)
plt.hist(sims, bins=10, rwidth= .9)  # arguments are passed to np.histogram
plt.title("Histogram of similarity values")
plt.show()

### 11) Save to a file

In [ ]:
#df_comp.head()
fn1 = 'sim_data_out_2999.csv'
df_comp.to_csv(fn1, encoding='utf-8' )